In [3]:
import numpy as np
import pandas as pd

np.random.seed(42)

def generate_hurricane_samples(label, n, center):
    return pd.DataFrame({
        "sea_surface_temperature":    np.random.normal(center["sst"], 0.8, n),
        "ocean_heat_content":         np.random.normal(center["ohc"], 10, n),
        "mid_level_humidity":         np.random.normal(center["humidity"], 8, n),
        "vertical_wind_shear":        np.random.normal(center["shear"], 3, n),
        "potential_vorticity":        np.random.normal(center["vort"], 0.4, n),
        "hurricane_binary":           label
    })

# --- Archetypes ---
atlantic_warm_blob = generate_hurricane_samples(1, 1200, {
    "sst": 29.5, "ohc": 110, "humidity": 75,
    "shear": 4, "vort": 1.8
})

saharan_suppressed = generate_hurricane_samples(0, 900, {
    "sst": 27, "ohc": 70, "humidity": 40,
    "shear": 18, "vort": 0.5
})

pre_spin_wave = generate_hurricane_samples(1, 800, {
    "sst": 28.5, "ohc": 90, "humidity": 65,
    "shear": 10, "vort": 1.2
})

dead_zone = generate_hurricane_samples(0, 700, {
    "sst": 26, "ohc": 50, "humidity": 35,
    "shear": 20, "vort": 0.4
})

mixed_noise = generate_hurricane_samples(1, 500, {
    "sst": 29, "ohc": 95, "humidity": 60,
    "shear": 9, "vort": 1.1
})

# --- Final Assembly ---
data = pd.concat([
    atlantic_warm_blob,
    saharan_suppressed,
    pre_spin_wave,
    dead_zone,
    mixed_noise
])
data = data.clip(lower=0).sample(frac=1, random_state=42).reset_index(drop=True)
data.to_csv("dataset/hurricane_data.csv", index=False)

print("HurricaneNet dataset generated:", data.shape)

HurricaneNet dataset generated: (4100, 6)


In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from sklearn.model_selection import train_test_split
from tensorflow.keras.saving import register_keras_serializable
from tensorflow.keras import layers, models, backend as K

data = pd.read_csv("dataset/hurricane_data.csv")
X = data.drop("hurricane_binary", axis=1).astype("float32")
y = data["hurricane_binary"].astype("float32")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.3, random_state=42
)

@register_keras_serializable()
class SSTAmplifier(tf.keras.layers.Layer):
    def __init__(self, threshold=28.0, scale=0.1, **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold
        self.scale = scale

    def call(self, inputs):
        sst = inputs[:, 0]
        factor = tf.sigmoid((sst - self.threshold) * self.scale)
        mod = 1.0 + 0.3 * factor
        return tf.expand_dims(mod, -1)

@register_keras_serializable()
class ShearSuppressor(tf.keras.layers.Layer):
    def __init__(self, threshold=14.0, scale=0.2, **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold
        self.scale = scale

    def call(self, inputs):
        shear = inputs[:, 3]
        suppress = tf.sigmoid((self.threshold - shear) * self.scale)
        mod = 1.0 - 0.25 * suppress
        return tf.expand_dims(mod, -1)

@register_keras_serializable()
class VorticityActivator(tf.keras.layers.Layer):
    def __init__(self, threshold=1.2, scale=1.0, **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold
        self.scale = scale

    def call(self, inputs):
        vort = inputs[:, 4]
        activate = tf.sigmoid((vort - self.threshold) * self.scale)
        mod = 1.0 + 0.2 * activate
        return tf.expand_dims(mod, -1)

@register_keras_serializable()
class ModulationMixer(tf.keras.layers.Layer):
    def call(self, inputs):
        sst_mod, shear_mod, vort_mod = inputs
        product = sst_mod * shear_mod * vort_mod
        smooth = 1.0 + 0.25 * tf.tanh(product - 1.0)
        return smooth

# --- Input Layer ---
input_layer = layers.Input(shape=(5,), name="hurricane_inputs")

x = layers.BatchNormalization()(input_layer)
x1 = layers.Dense(64, activation="relu")(x)
x2 = layers.Dense(32, activation="relu")(x1)
x3 = layers.Dense(16, activation="relu")(x2)
base_logits = layers.Dense(1)(x3)

sst_mod    = SSTAmplifier()(input_layer)
shear_mod  = ShearSuppressor()(input_layer)
vort_mod   = VorticityActivator()(input_layer)
mod_strength = ModulationMixer()([sst_mod, shear_mod, vort_mod])

combined_logits = layers.Add()([base_logits, mod_strength])
final_output = layers.Activation("sigmoid")(combined_logits)
model = models.Model(inputs=input_layer, outputs=final_output)
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False, label_smoothing=0.05),
    metrics=["accuracy"]
)

# --- Train ---
early_stop = callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
model.fit(X_train, y_train, validation_split=0.2, epochs=15, batch_size=16, callbacks=[early_stop])

# --- Evaluate & Save ---
loss, acc = model.evaluate(X_test, y_test)
print(f"HurricaneNet Accuracy: {acc:.4f}")
model.save("models/HurricaneNet.h5")
import tf2onnx
model_proto, _ = tf2onnx.convert.from_keras(model)
with open(f"models/ONNX/HurricaneNet.onnx", "wb") as f:
    f.write(model_proto.SerializeToString())


Epoch 1/15
144/144 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - accuracy: 0.7297 - loss: 0.4759 - val_accuracy: 0.6237 - val_loss: 1.3863
Epoch 2/15
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9804 - loss: 0.1697 - val_accuracy: 0.9791 - val_loss: 0.1718
Epoch 3/15
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9751 - loss: 0.1698 - val_accuracy: 0.9930 - val_loss: 0.1353
Epoch 4/15
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.9882 - loss: 0.1454 - val_accuracy: 0.9948 - val_loss: 0.1312
Epoch 5/15
144/144 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.9794 - loss: 0.1596 - val_accuracy: 0.9983 - val_loss: 0.1302
Epoch 6/15
144/144 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9851 - loss: 0.1540 - val_accuracy: 0.9965 - val_loss: 0.1321
Epoch 7/15
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9699 - loss: 0.1790 - val_accuracy: 0.9983 - val_loss: 0.1287
Epoch 8/15
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9831 - loss: 0.1570 - val_accura

HurricaneNet Accuracy: 0.9967




ERROR:tf2onnx.tfonnx:rewriter <function rewrite_constant_fold at 0x00000201D1353E20>: exception `np.cast` was removed in the NumPy 2.0 release. Use `np.asarray(arr, dtype=dtype)` instead.


In [6]:
import numpy as np
import pandas as pd

scenarios = [
    {
        "label": "🟠 Atlantic Warm Blob",
        "features": [29.7, 115, 78, 3.5, 2.0],
        "expected": "Hurricane"
    },
    {
        "label": "🟤 Saharan Air Layer Suppression",
        "features": [27.1, 68, 42, 19, 0.6],
        "expected": "No Hurricane"
    },
    {
        "label": "🌊 Tropical Wave Pre-Spin",
        "features": [28.6, 93, 67, 9.5, 1.4],
        "expected": "Hurricane"
    },
    {
        "label": "🌫️ Oceanic Dead Zone",
        "features": [26.3, 48, 38, 21, 0.5],
        "expected": "No Hurricane"
    },
    {
        "label": "🌬️ Shear-Cut Anomaly",
        "features": [29.0, 98, 62, 17, 1.2],
        "expected": "Possibly Hurricane"
    },
    {
        "label": "🌡️ Hot SST / Low Vorticity",
        "features": [30.1, 120, 75, 6, 0.3],
        "expected": "Edge Case"
    }
]

# --- Run predictions ---
print("\n🌀 HurricaneNet Scenario Evaluation:\n")
for case in scenarios:
    x = np.array(case["features"], dtype="float32").reshape(1, -1)
    pred = model(x).numpy()[0][0]
    verdict = (
        "Hurricane" if pred > 0.55 else
        "Possibly Hurricane" if 0.4 < pred <= 0.55 else
        "No Hurricane"
    )

    print(f"{case['label']}")
    print(f"  ➤ Features      : {case['features']}")
    print(f"  ➤ Predicted     : {verdict} (confidence: {pred:.2f})")
    print(f"  ➤ Expected      : {case['expected']}\n")


🌀 HurricaneNet Scenario Evaluation:

🟠 Atlantic Warm Blob
  ➤ Features      : [29.7, 115, 78, 3.5, 2.0]
  ➤ Predicted     : Hurricane (confidence: 0.98)
  ➤ Expected      : Hurricane

🟤 Saharan Air Layer Suppression
  ➤ Features      : [27.1, 68, 42, 19, 0.6]
  ➤ Predicted     : No Hurricane (confidence: 0.04)
  ➤ Expected      : No Hurricane

🌊 Tropical Wave Pre-Spin
  ➤ Features      : [28.6, 93, 67, 9.5, 1.4]
  ➤ Predicted     : Hurricane (confidence: 0.97)
  ➤ Expected      : Hurricane

🌫️ Oceanic Dead Zone
  ➤ Features      : [26.3, 48, 38, 21, 0.5]
  ➤ Predicted     : No Hurricane (confidence: 0.02)
  ➤ Expected      : No Hurricane

🌬️ Shear-Cut Anomaly
  ➤ Features      : [29.0, 98, 62, 17, 1.2]
  ➤ Predicted     : Hurricane (confidence: 0.96)
  ➤ Expected      : Possibly Hurricane

🌡️ Hot SST / Low Vorticity
  ➤ Features      : [30.1, 120, 75, 6, 0.3]
  ➤ Predicted     : Hurricane (confidence: 0.99)
  ➤ Expected      : Edge Case

